# Dot detection using DAOStarFinder

In [1]:
#custom function
from daostarfinder_dotdetection import *
from util import pil_imread
#enhance figure display
%config InlineBackend.figure_format = 'retina'

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False, add_trace = True, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel-1])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel-1], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel-1],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        if len(tiff.shape) == 3:
            tiff = tiff.reshape(1,tiff.shape[0],tiff.shape[1],tiff.shape[2])
        print("shape =", tiff.shape)
        #plot
        for z in range(len(tiff[:,channel-1])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel-1], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel-1],add_trace=add_trace, zmax=zmax)

In [ ]:
import time
#start time
start = time.time()
hyb = 0
pos = 0
#image sources
img_src = f"/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/pre_processed_images/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif"
img_raw = f'/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/dapi_aligned/fiducial_aligned/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
raw_src=None

#img_src: path to image
#HybCycle: which hybcycle are we looking at
#size_cutoff: number of standard deviation away from mean size area
#threshold: absolute pixel intensity the spot must be greater than
#channel: which channel you want to analyze

dots = dot_detection(img_src, HybCycle=hyb, size_cutoff=4, threshold=0.01,channel=1, num_channels=4)
print(f"This task took {(time.time() - start)/60} minutes")

In [ ]:
# plot dots on top of image
plot_3d_locs_on_2d_image(dots,None, tiff=None, channel=1, raw_src = img_raw, 
                         raw_image = True, add_trace=False, zmax=5000)

In [ ]:
# plot dots on top of image
tiff = pil_imread(img_src, swapaxes=True)
plot_3d_locs_on_2d_image(dots,None, tiff=tiff, channel=1, raw_src = None, 
                         raw_image = False, add_trace=False, zmax=0.1)

# Quick colocalization check

For colocalization between 1st and last hyb. Use the "hyb_coloc" scripts in colocalization_files. The necessary batch files are there for analyzing multiple channels and pos in parallel.